# 00 Single-Factor Regression (Micro)

Single-factor log-log regression on county data; interpret coefficients.


## Table of Contents
- [Load census data](#load-census-data)
- [Build log variables](#build-log-variables)
- [Fit OLS + HC3](#fit-ols-hc3)
- [Interpretation](#interpretation)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Regression is the bridge between statistics and ML. You will learn:
- single-factor vs multi-factor interpretation,
- robust standard errors,
- coefficient stability and multicollinearity.


## Prerequisites (Quick Self-Check)
- Completed Parts 00–01 (foundations + data).
- Basic algebra comfort (reading coefficient tables, units).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating coefficients as causal without a causal design.
- Ignoring multicollinearity (unstable coefficients).

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/02_regression/00_single_factor_regression_micro.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/02_regression/00_single_factor_regression_micro.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Fit a single-factor log-log regression on county data and interpret the coefficient like an elasticity.

Example question:
- "Across counties, how does rent scale with income?"

This is **not** a causal claim by default. It's a structured description of a relationship.



## Primer: `statsmodels` vs `scikit-learn` (inference vs prediction)

This repo uses both libraries because they serve different goals:

- **Prediction (ML):** optimize out-of-sample accuracy → `scikit-learn`
- **Inference (econometrics):** interpret coefficients + quantify uncertainty → `statsmodels`

### Minimal `statsmodels` OLS pattern

```python
import statsmodels.api as sm

# X: DataFrame of features, y: Series target
Xc = sm.add_constant(X, has_constant="add")  # add intercept
res = sm.OLS(y, Xc).fit()
print(res.summary())
```

**Expected output / sanity check**
- a table with `coef`, `std err`, `t`, `P>|t|`, and a CI column
- coefficient names match your column names

### What you are looking at in `res.summary()`

- **coef**: $\\hat\\beta$ (estimated effect in the model)
- **std err**: estimated uncertainty $\\widehat{SE}(\\hat\\beta)$
- **t**: $\\hat\\beta / \\widehat{SE}(\\hat\\beta)$
- **P>|t|**: p-value for $H_0: \\beta=0$ (conditional on assumptions)
- **[0.025, 0.975]**: 95% confidence interval

### Robust standard errors (change uncertainty, not coefficients)

```python
# Cross-section heteroskedasticity
res_hc3 = res.get_robustcov_results(cov_type="HC3")

# Time series autocorrelation + heteroskedasticity
res_hac = res.get_robustcov_results(cov_type="HAC", cov_kwds={"maxlags": 4})
```

### Common pitfalls (and quick fixes)

- **Forgetting the intercept**
  - Fix: always `add_constant`.
- **Wrong SE for time series**
  - Fix: use HAC when residuals are autocorrelated.
- **Treating p-values as causal proof**
  - Fix: write the identification assumption; otherwise interpret as association.
- **Mixing prediction and inference**
  - Fix: use `sklearn` pipelines + time splits for prediction; use `statsmodels` for coefficient uncertainty.


## Primer: Hypothesis testing (p-values, t-stats, confidence intervals)

You will see p-values, t-statistics, and confidence intervals in regression output (especially `statsmodels`). This primer gives you the minimum to interpret them correctly.

### The objects (plain language)

- **Null hypothesis** $H_0$: the default claim (often “no effect”).
- **Alternative** $H_1$: the claim you consider if the data looks inconsistent with $H_0$.
- **Test statistic**: “how far” your estimate is from the null, in uncertainty units.
- **p-value**: probability (under the null *and model assumptions*) of seeing a test statistic at least as extreme as observed.
- **Confidence interval (CI)**: a range of parameter values consistent with the data under assumptions.

### What a p-value is NOT

- Not the probability $H_0$ is true.
- Not the probability the model is correct.
- Not a measure of economic importance.

### Regression t-test intuition

In OLS, a common test is $H_0: \\beta_j = 0$.

$$
t_j = \\frac{\\hat\\beta_j}{\\widehat{SE}(\\hat\\beta_j)}
$$

If you change your SE estimator (HC3/HAC/cluster), you change $\\widehat{SE}$ and therefore the p-value, even if the coefficient stays the same.

### Expected output / what you should look at in `res.summary()`

- `coef`: effect size (in model units)
- `std err`: uncertainty
- CI columns: magnitude + uncertainty together

### Common pitfalls in this project

- Macro time series often have autocorrelation → naive SE too small → use HAC when interpreting p-values.
- Multiple testing/spec-search can produce small p-values by chance.
- Predictive success ≠ causal interpretation.

### Tiny demo (toy; not project data)

```python
import numpy as np
import pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(0)
n = 300
x = rng.normal(size=n)
y = 1.0 + 0.5 * x + rng.normal(scale=1.0, size=n)

df = pd.DataFrame({"y": y, "x": x})
X = sm.add_constant(df[["x"]])
res = sm.OLS(df["y"], X).fit()
print(res.summary())
```


<a id="load-census-data"></a>
## Load census data

### Goal
Load a county-level dataset created in the Census notebook.

If you haven't run the fetch notebook, use the bundled sample.



### Your Turn (1): Load processed county data (or sample)


In [ ]:
import pandas as pd

year = 2022  # TODO: set to the year you fetched
path = PROCESSED_DIR / f'census_county_{year}.csv'

if path.exists():
    df = pd.read_csv(path)
else:
    df = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')

df.head()



### Your Turn (2): Inspect schema


In [ ]:
# TODO: Inspect columns and dtypes.
# Identify which columns represent:
# - income
# - rent
print(df.columns.tolist())
print(df.dtypes)
...



### Checkpoint (required columns)


In [ ]:
# TODO: Confirm the starter variables exist.
# If you used different ACS vars, update these names.
assert 'B19013_001E' in df.columns  # median household income
assert 'B25064_001E' in df.columns  # median gross rent
...



<a id="build-log-variables"></a>
## Build log variables

### Goal
Build a clean modeling table with:
- `log_income = log(income)`
- `log_rent = log(rent)`

Log-log regression is common in economics because it turns multiplicative relationships into additive ones.



### Your Turn (1): Clean and log-transform


In [ ]:
import numpy as np

# Raw variables
income = pd.to_numeric(df['B19013_001E'], errors='coerce')
rent = pd.to_numeric(df['B25064_001E'], errors='coerce')

# TODO: Drop non-positive values before taking logs
mask = (income > 0) & (rent > 0)
df_m = pd.DataFrame({
    'income': income[mask],
    'rent': rent[mask],
}).dropna()

# TODO: Create log variables
df_m['log_income'] = ...
df_m['log_rent'] = ...

df_m[['log_income', 'log_rent']].head()



### Your Turn (2): Visualize the relationship


In [ ]:
import matplotlib.pyplot as plt

# TODO: Make a scatter plot of log_income vs log_rent.
# Tip: use alpha=0.2 for dense plots.
...



<a id="fit-ols-hc3"></a>
## Fit OLS + HC3

### Goal
Fit an OLS regression and compute HC3 robust SE (common default for cross-sectional data).

Model:
$$\log(rent_i) = \beta_0 + \beta_1 \log(income_i) + \varepsilon_i$$



### Your Turn (1): Fit OLS + HC3


In [ ]:
from src import econometrics

# TODO: Fit OLS with HC3 robust SE using the helper.
res = econometrics.fit_ols_hc3(df_m, y_col='log_rent', x_cols=['log_income'])
print(res.summary())



### Your Turn (2): Extract the slope and interpret it


In [ ]:
# TODO: Extract coefficient and CI for log_income.
beta = float(res.params['log_income'])
ci = res.conf_int().loc['log_income'].tolist()

print('beta:', beta)
print('95% CI:', ci)

# Interpretation prompt:
# In a log-log model, beta is approximately an elasticity.
# Example: a 1% increase in income is associated with about beta% higher rent.
# TODO: Compute the implied change for a 10% income increase.
pct_income = 10.0
approx_pct_rent = ...
approx_pct_rent



<a id="interpretation"></a>
## Interpretation

Write a short interpretation (5-8 sentences) that answers:
- What is the estimated relationship?
- Is it statistically distinguishable from 0 (given assumptions)?
- Is it economically large?
- What would have to be true for a causal interpretation?



### Your Turn: Write your interpretation


In [ ]:
# TODO: Write your interpretation as a Python multiline string.
notes = """
...
"""
print(notes)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After fitting the model, validate the data and results.
# This is cross-sectional data (no time splits needed).
# Example checks (adjust variable names):
# assert df_m.shape[0] > 100, "Too few observations after filtering"
# assert not df_m[['log_income', 'log_rent']].isna().any().any(), "NaNs remain"
# assert 0 < beta < 2, f"Elasticity {beta} outside plausible range"
# assert ci[0] < ci[1], "CI bounds are inverted"
...

## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Load census data</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_single_factor_regression_micro — Load census data
import numpy as np
import pandas as pd

df = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')
df.head()
```

</details>

<details><summary>Solution: Build log variables</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_single_factor_regression_micro — Build log variables
df = df.copy()
df = df[(df['B19013_001E'] > 0) & (df['B25064_001E'] > 0)].copy()
df['log_income'] = np.log(df['B19013_001E'].astype(float))
df['log_rent'] = np.log(df['B25064_001E'].astype(float))
```

</details>

<details><summary>Solution: Fit OLS + HC3</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_single_factor_regression_micro — Fit OLS + HC3
from src import econometrics

res = econometrics.fit_ols_hc3(df, y_col='log_rent', x_cols=['log_income'])
print(res.summary())
```

</details>

<details><summary>Solution: Interpretation</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_single_factor_regression_micro — Interpretation
# In a log-log model, the slope is an elasticity-style interpretation:
# a 1% increase in income is associated with ~beta% increase in rent (under assumptions).
```

</details>

